# HoVerNet Inference Demo

This notebook runs nuclear segmentation on your histopathology image using HoVerNet.

**Two preprocessing fixes are applied automatically:**
1. **Color correction** — Your image has Cyan staining (High B+G). The model expects H&E/Purple (High B+R). We swap R and G channels.
2. **Magnification correction** — Your image is ~10x but the PanNuke model expects 40x. We upscale each tile 4x before inference.

In [ ]:
import os
import sys
sys.path.insert(0, '.')

INPUT_IMAGE = "test_input/test_image2.jpg"
OUTPUT_DIR  = "inference_output"

from run_tiled_inference import run_tiled_inference

print("Running tiled inference (this takes ~20 min on CPU)...")
inst_map, centroids = run_tiled_inference(
    INPUT_IMAGE, 
    OUTPUT_DIR, 
    scale_factor=4,   # 10x -> 40x
    tile_size=256,
    overlap=32
)
print(f"Done! Detected {len(centroids)} nuclei.")

## View Results

Three output files are generated:
- `overlay_nuclei.png` — Nuclear contours on original image
- `overlay_cells.png` — Dilated cell boundaries for visual verification
- `summary.png` — Side-by-side comparison with probability heatmap

In [ ]:
import matplotlib.pyplot as plt
import cv2

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

nuclei = cv2.imread(f"{OUTPUT_DIR}/results/overlay_nuclei.png")
nuclei = cv2.cvtColor(nuclei, cv2.COLOR_BGR2RGB)
axes[0].imshow(nuclei)
axes[0].set_title("Nuclear Segmentation", fontsize=14)
axes[0].axis('off')

cells = cv2.imread(f"{OUTPUT_DIR}/results/overlay_cells.png")
cells = cv2.cvtColor(cells, cv2.COLOR_BGR2RGB)
axes[1].imshow(cells)
axes[1].set_title("Cell-Level Visualization", fontsize=14)
axes[1].axis('off')

plt.tight_layout()
plt.show()